In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder 
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_predict
from sklearn import preprocessing

from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier ,AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn import tree
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
st = pd.read_csv("../input/xAPI-Edu-Data.csv")

In [ ]:
st.Class =st.Class.astype('category')
class_order = {'H':3, 'M':2, 'L':1}
st.Class =st.Class.map(class_order)

L1-based feature selection¶
---------------------------

Linear models penalized with the L1 norm have sparse solutions: many of their estimated coefficients are zero. When the goal is to reduce the dimensionality of the data to use with another classifier, they can be used along with feature_selection.SelectFromModel to select the non-zero coefficients. In particular, sparse estimators useful for this purpose are the linear_model.Lasso for regression, and of linear_model.LogisticRegression and svm.LinearSVC for classification: With SVMs and logistic-regression, the parameter C controls the sparsity: the smaller C the fewer features selected. With Lasso, the higher the alpha parameter, the fewer features selected.

In [ ]:
def Feature_Selector(data):
    
    st1 =data.drop('Class',axis=1)
    st2= data['Class']
    st1 = pd.get_dummies(st1)
    st1 = pd.DataFrame(preprocessing.scale(st1) ,columns=st1.columns )
    st1_columns = st1.columns
    lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(st1, st2)
    selector =  SelectFromModel(lsvc, prefit=True)
    selector.transform(st1)
    labels = [st1_columns[x] for x in selector.get_support(indices=True)]
    st1 = pd.DataFrame(selector.transform(st1),columns=labels)
    data = pd.concat([st1,st2],axis=1)
    return data

In [ ]:
red =Feature_Selector(st)
red.head()

## With Reduced Features ##

In [ ]:
X, y = red.drop('Class',axis=1) , red.Class
#Split the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

In [ ]:
#Random Forest Hyperparameter selection n_estimator
accr = {}
for i in range(1,100):
    clf = RandomForestClassifier(n_estimators = i,random_state =1)
    clf = clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    scr = metrics.accuracy_score(y_test, y_pred)
    accr[i] = scr
    
scores = []
for i in accr:
    scores.append(accr[i])
    
for i in accr:
    if accr[i] == max(scores):
        print(i,accr[i])

In [ ]:
#Bagging Hyperparameter selection n_estimator
accr = {}
for i in range(1,250):
    clf = BaggingClassifier( n_estimators=i, max_samples=1.0, max_features=1.0,random_state=1)
    clf = clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    scr = metrics.accuracy_score(y_test, y_pred)
    accr[i] = scr
    
scores = []
for i in accr:
    scores.append(accr[i])
    
for i in accr:
    if accr[i] == max(scores):
        print(i,accr[i])

In [ ]:
clf5  = RandomForestClassifier(n_estimators = 73,random_state=1)
clf6  =BaggingClassifier( n_estimators=47, max_samples=1.0, max_features=1.0,random_state=1)
classifiers = { "Random Forest": clf5,"Bagging": clf6}

In [ ]:
for clf in classifiers: 
    predicted = cross_val_predict(classifiers[clf],  X_train, y_train, cv=10)
    train_acc = metrics.accuracy_score(y_train, predicted) 
    c = classifiers[clf].fit(X_train,y_train)
    y_pred = c.predict(X_test)
    test_acc = metrics.accuracy_score(y_test, y_pred)
    print(clf)
    print("\n Accuracy on training set: ",train_acc,
      "\n Accuracy on test set: ",test_acc
         ,"\n-------------*----------------")